#Mount Drive, Auto-reload setup

In [28]:
# !pip install -q transformers datasets huggingface_hub evaluate sentencepiece scikit-learn imbalanced-learn accelerate torch pandas matplotlib

In [29]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"
os.environ["WANDB_SILENT"] = "true"

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import os

GOOGLE_DRIVE_PATH = os.path.join('/content', 'drive', 'My Drive', 'ColabNotebooks', 'DL','Deep Learning Project Full','DL Team Project', '20251126 22.47 DL Team Project - Code, Files, Structure')
print(os.listdir(GOOGLE_DRIVE_PATH))

['environment.yml', 'Copy of KP DL Project Colab Notebook.ipynb', 'Archive or Obsolete', 'configs', 'Obsolete - models', 'src', 'data', '__pycache__', 'outputs', '20251126 22.47 KP DL Project Colab Notebook.ipynb']


In [32]:
import sys

sys.path.append(GOOGLE_DRIVE_PATH)

In [33]:
# if running locally set GOOGLE PATH
import sys
if 'google.colab' in sys.modules:
  print(f'Running in google colab. Our path is `{GOOGLE_DRIVE_PATH}`')
else:
  GOOGLE_DRIVE_PATH = '.'
  print('Running locally.')

Running in google colab. Our path is `/content/drive/My Drive/ColabNotebooks/DL/Deep Learning Project Full/DL Team Project/20251126 22.47 DL Team Project - Code, Files, Structure`


In [34]:
print("Current working directory:", os.getcwd())

Current working directory: /content/drive/My Drive/ColabNotebooks/DL/Deep Learning Project Full/DL Team Project/20251126 22.47 DL Team Project - Code, Files, Structure


In [35]:
# ch dir to project folder
os.chdir(GOOGLE_DRIVE_PATH)

In [36]:
print("Current working directory:", os.getcwd())

Current working directory: /content/drive/My Drive/ColabNotebooks/DL/Deep Learning Project Full/DL Team Project/20251126 22.47 DL Team Project - Code, Files, Structure


In [37]:
# import imp

In [38]:
# %load_ext autoreload
# # %reload_ext autoreload
# %autoreload 2

In [39]:
# !pip install -q --upgrade transformers datasets huggingface_hub

In [40]:
# optional: check torch & CUDA
import torch
print("torch:", torch.__version__, "cuda_available:", torch.cuda.is_available())

torch: 2.9.0+cu126 cuda_available: False


#Read ME

In [47]:
'''
Keep res_prep and res_train variables in the notebook. They hold datasets and trainer
so you can inspect easily.

To get smoother loss curves, set logging_steps smaller (e.g., 20). But smaller logs
slow training a little.

If you restart the kernel, reload tokenized datasets with:
from datasets import load_from_disk
train_tok = load_from_disk(res_prep["train_tok_path"])
val_tok = load_from_disk(res_prep["val_tok_path"])
test_tok = load_from_disk(res_prep["test_tok_path"])

Use small subset_sample during development (2k–10k) and increase later.

If langdetect throws occasional errors, try running again — or remove language filter for debugging.
'''

'\nKeep res_prep and res_train variables in the notebook. They hold datasets and trainer\nso you can inspect easily.\n\nTo get smoother loss curves, set logging_steps smaller (e.g., 20). But smaller logs\nslow training a little.\n\nIf you restart the kernel, reload tokenized datasets with:\nfrom datasets import load_from_disk\ntrain_tok = load_from_disk(res_prep["train_tok_path"])\nval_tok = load_from_disk(res_prep["val_tok_path"])\ntest_tok = load_from_disk(res_prep["test_tok_path"])\n\nUse small subset_sample during development (2k–10k) and increase later.\n\nIf langdetect throws occasional errors, try running again — or remove language filter for debugging.\n'

In [48]:
'''
src/common_data_prep.py — common data-prep utilities (steps 1–13). Teammates call this to get preprocessed & tokenized datasets. It uses src/utils.py but does not change it.

src/model_BERT.py — training & evaluation code (only training/eval). This file does not hardcode MODEL_CONFIGS. Instead it accepts MODEL_NAME, num_labels, and a training_args dict from the notebook so teammates can choose any model and hyperparams in the notebook. You (or teammates) can create their own model_xxx.py with the same train_and_evaluate() signature.

Two notebook cells: one to run data prep and one to run training & evaluation. The notebook controls MODEL_NAME, batch sizes, epochs, etc.

Everything is simple, new-student friendly, and copy-paste ready.
'''

'\nsrc/common_data_prep.py — common data-prep utilities (steps 1–13). Teammates call this to get preprocessed & tokenized datasets. It uses src/utils.py but does not change it.\n\nsrc/model_BERT.py — training & evaluation code (only training/eval). This file does not hardcode MODEL_CONFIGS. Instead it accepts MODEL_NAME, num_labels, and a training_args dict from the notebook so teammates can choose any model and hyperparams in the notebook. You (or teammates) can create their own model_xxx.py with the same train_and_evaluate() signature.\n\nTwo notebook cells: one to run data prep and one to run training & evaluation. The notebook controls MODEL_NAME, batch sizes, epochs, etc.\n\nEverything is simple, new-student friendly, and copy-paste ready.\n'

#**Prepare the pre-processed data **

(STRICTLY DO NOT RUN THIS CELL). Directly go to next section to import pre-processed data and continue from there to explore (EDA) the pre-processed data and then run models in next cells.

In [42]:
# '''
# (run this cell only if needed). Use the next cell directly to import the csv file called
# data_train.csv to load data and then run models.
# '''

# # # Cell A: data prep (team-safe)
# import os
# os.environ["WANDB_DISABLED"] = "true"

# from src.common_data_prep import prepare_data

# res_prep = prepare_data(
#     repo_id="McAuley-Lab/Amazon-Reviews-2023",
#     chosen_jsonl="raw/review_categories/All_Beauty.jsonl",
#     output_root="outputs",
#     data_root="data",
#     model_name_tag="All_Beauty_prep",
#     subset_sample=None,         # small for debugging; set None to use full oversampled train
#     downsample_target=None,     # or 200000
#     max_words=256,
#     langdetect_ok=True
# )

# print("Prep done. Tokenized saved in:", res_prep["run_outdir"])
# print("Train rows:", res_prep["info"]["train_rows"])


Saved CSV: data/raw_data.csv
After English filter (robust) rows: 578425
train_bal columns: ['rating', 'text', 'label3', 'label5', 'year', 'lang']


Map:   0%|          | 0/957702 [00:00<?, ? examples/s]

Saved CSV: data/data_train.csv


Map:   0%|          | 0/957702 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/957702 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/57837 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/57843 [00:00<?, ? examples/s]

Prep done. Tokenized saved in: outputs/All_Beauty_prep_20251128_005818
Train rows: 957702


#If you really want to look at the raw-data, then use below cells. Otherwise ignore this (recommended - to save your time) and move on to next section .

In [ ]:
#Import raw data
csv_path = f"{GOOGLE_DRIVE_PATH}/data/raw_data.csv"

# ---------------------------------------
# Load CSV
# ---------------------------------------
df_rawData = pd.read_csv(csv_path)

# ---------------------------------------
# Keep only needed columns (drop lang)
# ---------------------------------------
df_rawData = df_rawData.drop(columns=["lang"])   # remove lang column

#Just for Information: Number of rows in pre-processed data = 701,528



In [ ]:
df_rawData.head()

#Load Pre-Processed Data

In [ ]:
#Import pre-processed data
from sklearn.model_selection import train_test_split
csv_path = f"{GOOGLE_DRIVE_PATH}/data/data_train.csv"

# ---------------------------------------
# 3. Load CSV
# ---------------------------------------
df = pd.read_csv(csv_path)

# ---------------------------------------
# 4. Keep only needed columns (drop lang)
# ---------------------------------------
df = df.drop(columns=["lang"])   # remove lang column

#Just for Information: Number of rows in pre-processed data = 578,425

#For training & deubgging choose only 2000 or so samples subset
#Disable below line for full training sample set which has total 578,425 rows (or data points)
df = df.sample(n=2000, random_state=42)

# OR if you only want these columns:
# df = df[["text", "rating", "label3", "label5"]]

# ---------------------------------------
# 5. Stratify Split 80/10/10
# ---------------------------------------
strat_col = "label3"

train_bal, temp_df = train_test_split(
    df,
    test_size=0.20,
    random_state=42,
    stratify=df[strat_col]
)

val, test = train_test_split(
    temp_df,
    test_size=0.50,
    random_state=42,
    stratify=temp_df[strat_col]
)

print("Train:", train_bal.shape)
print("Val:", val.shape)
print("Test:", test.shape)

#Train & evaluate (set MODEL_NAME and hyperparams here) - Less Epochs used here to debug if the model is running fine or not

##**Train and then Print metrics (for less epochs training)**
Note: Be careful to choose correct Model and Hyper Parameters before you run below cells.

In [ ]:
# Cell B: train & evaluate
from src.model_BERT import train_and_evaluate

# Set model and hyperparams here (free to change in notebook)
MODEL_NAME = "distilbert-base-uncased"   # change to any HuggingFace model
num_labels = 3
training_args_overrides = {
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 2,
    "learning_rate": 2e-5,
    "logging_steps": 50
}



from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)
def tokenize_batch(batch):
  return tokenizer(batch["text"], truncation=True, max_length=256)
# remove text for train to save disk, keep for val/test
train_tok = train_bal.map(tokenize_batch, batched=True, remove_columns=["text", "rating"])
val_tok = val.map(tokenize_batch, batched=True)
test_tok = test.map(tokenize_batch, batched=True)

res_train = train_and_evaluate(
    train_tok=res_prep["train_tok"],   # in-memory tokenized datasets
    val_tok=res_prep["val_tok"],
    test_tok=res_prep["test_tok"],
    MODEL_NAME=MODEL_NAME,
    num_labels=num_labels,
    training_args_overrides=training_args_overrides,
    output_root="outputs",
    run_name_suffix="student_debug"
)

print("Train done. run_outdir:", res_train["run_outdir"])
print("Eval metrics:", res_train["eval"]["metrics"])


##Plot Curves (for less epochs training)

###General Plot Curve Code

In [ ]:
# # CELL: Plot loss vs epochs (use res_train)
# # Requires res_train variable from train_and_evaluate(...)
# from src.utils import plot_loss_vs_epochs
# import os

# run_outdir = res_train["run_outdir"]
# print("Reading logs from:", run_outdir)

# # Prefer the saved trainer_history.csv then fallback to trainer.state.log_history
# hist_csv = os.path.join(run_outdir, "trainer_history.csv")
# df_epochs = plot_loss_vs_epochs(trainer=res_train.get("trainer"), hist_csv=hist_csv, run_outdir=run_outdir)

# if df_epochs.empty:
#     print("No epoch-level data to plot. If you see only one epoch, try training with num_train_epochs>=2 to produce a curve.")
# else:
#     print("Epoch table:\n", df_epochs)


In [ ]:
# COMBINED CELL: plot loss + save loss CSV + classification report + confusion matrix
from src.utils import plot_loss_vs_epochs
import os, json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

run_outdir = res_train["run_outdir"]

# 1) Loss plot
hist_csv = os.path.join(run_outdir, "trainer_history.csv")
df_epochs = plot_loss_vs_epochs(trainer=res_train.get("trainer"), hist_csv=hist_csv, run_outdir=run_outdir)

# 2) Classification report + confusion matrix
trainer = res_train["trainer"]
test_tok = res_train["test_tok"]
pred_out = trainer.predict(test_tok)
y_true = pred_out.label_ids
y_pred = pred_out.predictions.argmax(axis=-1)

report = classification_report(y_true, y_pred, digits=4, output_dict=True)
with open(os.path.join(run_outdir, "classification_report_combined.json"), "w", encoding="utf-8") as fh:
    json.dump(report, fh, indent=2)
pd.DataFrame(report).transpose().to_csv(os.path.join(run_outdir, "classification_report_combined.csv"))

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
plt.figure(figsize=(6,6))
disp.plot(values_format='d')
plt.title("Confusion Matrix (test)")
plt.savefig(os.path.join(run_outdir, "confusion_matrix_combined.png"), bbox_inches='tight')
plt.show()

print("Saved loss plots and classification artifacts into:", run_outdir)


'''
Quick reminders (simple)

To get a proper curve, train for 2 or more epochs (set num_train_epochs >= 2).

The in-session cell saves trainer_state_log_history.json and loss_per_epoch.csv to the run folder. The post-session cell reads those files so you don’t need to retrain.

If auto-find picks the wrong folder, paste run_outdir manually in Cell B or C.


'''

'''
Below is a tiny single-line notebook cell that auto-selects the most recent run folder under outputs/, prints its path, and loads both loss_per_epoch.csv (into df) and a trainer log JSON (trainer_state_log_history.json or trainer_state.json) into log. It prints how many rows/entries were loaded so you know it worked.
'''

In [ ]:
run_outdir = max([d for d in __import__('glob').glob('outputs/*') if __import__('os').path.isdir(d)], key=__import__('os').path.getmtime); print("run_outdir:", run_outdir); import json, pandas as pd, os; csv_path=os.path.join(run_outdir,'loss_per_epoch.csv'); df = pd.read_csv(csv_path) if os.path.exists(csv_path) else pd.DataFrame(); log_path=os.path.join(run_outdir,'trainer_state_log_history.json'); log = json.load(open(log_path)) if os.path.exists(log_path) else (json.load(open(os.path.join(run_outdir,'trainer_state.json'))) if os.path.exists(os.path.join(run_outdir,'trainer_state.json')) else None); print("loss_per_epoch rows:", len(df), " | log entries:", (len(log) if log else 0))


**If neither JSON exists, log will be None.**

###Cell A — run this immediately after training (session active) — uses the trainer in memory and saves useful files into the run folder.

In [ ]:
# #Cell 1 — In-session plotting (run right after training)
# # CELL: Plot loss vs epochs now (session active)
# # Requires: res_train from train_and_evaluate(...) containing 'trainer' and 'run_outdir'
# from src.plot_utils import plot_loss_vs_epochs, parse_trainer_log_history
# import json, os

# if "res_train" not in globals():
#     raise ValueError("res_train not found. Run training cell first and keep res_train in session.")

# trainer = res_train["trainer"]
# run_outdir = res_train["run_outdir"]
# print("Using run_outdir:", run_outdir)

# # Save trainer.state.log_history to JSON (safe for post-session)
# try:
#     log_history = getattr(trainer.state, "log_history", None)
#     if log_history:
#         os.makedirs(run_outdir, exist_ok=True)
#         with open(os.path.join(run_outdir, "trainer_state_log_history.json"), "w", encoding="utf-8") as fh:
#             json.dump(log_history, fh, indent=2)
#         print("Saved trainer_state_log_history.json to run folder.")
#     else:
#         print("trainer.state.log_history is empty or not found.")
# except Exception as e:
#     print("Warning: could not save trainer_state.log_history:", e)

# # Prefer trainer_history.csv if trainer saved it; else use trainer.state.log_history
# hist_csv = os.path.join(run_outdir, "trainer_history.csv")
# df_epochs = plot_loss_vs_epochs(trainer=trainer, hist_csv=hist_csv, run_outdir=run_outdir, show_plot=True)

# if df_epochs.empty:
#     print("No epoch-level losses found. If you trained only 1 epoch you'll see a single point. Train >=2 epochs for a curve.")
# else:
#     print("Epoch table (last train loss and eval loss per epoch):")
#     print(df_epochs)
#     # Loss CSV already saved by the function as loss_per_epoch.csv


In [ ]:
# CELL A: In-session plotting (run right after training)
# Paste and run this cell after your training cell that produced `res_train`.
# It reads trainer.state.log_history, saves JSON+CSV for later, and plots loss curves.

import os
import json
from src.plot_utils import plot_loss_vs_epochs, parse_trainer_log_history

# check res_train
if "res_train" not in globals():
    raise ValueError("res_train not found. Run training (train_and_evaluate) first and keep res_train in the session.")

trainer = res_train["trainer"]
run_outdir = res_train["run_outdir"]
print("Using run_outdir:", run_outdir)

# 1) Save trainer.state.log_history to JSON for later use (post-session)
try:
    log_history = getattr(trainer.state, "log_history", None)
    if log_history:
        os.makedirs(run_outdir, exist_ok=True)
        json_path = os.path.join(run_outdir, "trainer_state_log_history.json")
        with open(json_path, "w", encoding="utf-8") as fh:
            json.dump(log_history, fh, indent=2)
        print("Saved trainer_state_log_history.json to:", json_path)
    else:
        print("trainer.state.log_history is empty or missing.")
except Exception as e:
    print("Warning: could not save trainer_state.log_history:", e)

# 2) Prefer trainer_history.csv if present, else parse trainer.state.log_history
hist_csv = os.path.join(run_outdir, "trainer_history.csv")
df_epochs = plot_loss_vs_epochs(trainer=trainer, hist_csv=hist_csv, run_outdir=run_outdir, show_plot=True)

if df_epochs.empty:
    print("No epoch-level losses found. If you trained only 1 epoch you'll see a single point. Train >=2 epochs for a curve.")
else:
    print("Epoch table (last training loss and eval loss per epoch):")
    print(df_epochs)
    # loss_per_epoch.csv saved by function when run_outdir provided


###Cell B — run later after session restart — finds the run folder (or use the path you saved) and plots from saved CSV / saved trainer_state JSON without retraining.

In [ ]:
# #Cell 2 — Post-session plotting (after restart; load from disk)

# # CELL: Plot loss vs epochs after session restart (no trainer in memory)
# # Provide run_outdir or auto-find the latest folder under outputs/
# from src.plot_utils import plot_loss_vs_epochs
# import os, glob

# # Option A: paste your run_outdir path here (preferred)
# # run_outdir = "outputs/your_model_lr..._2025xxxx_xxxxxx"
# run_outdir = None

# # Option B: auto-find the most recent run folder under outputs/
# if run_outdir is None:
#     base = "outputs"
#     candidates = [d for d in glob.glob(os.path.join(base, "*")) if os.path.isdir(d)]
#     if not candidates:
#         raise FileNotFoundError("No run folders found under 'outputs/'. Provide run_outdir manually.")
#     # pick newest by modification time
#     run_outdir = max(candidates, key=os.path.getmtime)
#     print("Auto-selected latest run folder:", run_outdir)

# # Prefer the saved epoch CSV if present
# epoch_csv = os.path.join(run_outdir, "loss_per_epoch.csv")
# if os.path.exists(epoch_csv):
#     print("Found loss_per_epoch.csv — plotting from that.")
#     df = plot_loss_vs_epochs(trainer=None, hist_csv=epoch_csv, run_outdir=run_outdir, show_plot=True)
# else:
#     # Try the previously saved trainer_state_log_history.json or trainer_state.json
#     json_candidates = [
#         os.path.join(run_outdir, "trainer_state_log_history.json"),
#         os.path.join(run_outdir, "trainer_state.json"),
#     ]
#     found = False
#     for jpath in json_candidates:
#         if os.path.exists(jpath):
#             print("Found", jpath, "- parsing and plotting.")
#             df = plot_loss_vs_epochs(trainer=None, hist_csv=None, run_outdir=run_outdir, show_plot=True)
#             found = True
#             break
#     if not found:
#         # Try to parse any json under run_outdir
#         parsed = False
#         for root, _, files in os.walk(run_outdir):
#             for f in files:
#                 if f.lower().endswith(".json"):
#                     cand = os.path.join(root, f)
#                     try:
#                         # plot_loss_vs_epochs will scan these files for log_history
#                         df = plot_loss_vs_epochs(trainer=None, hist_csv=None, run_outdir=run_outdir, show_plot=True)
#                         parsed = True
#                         break
#                     except:
#                         continue
#             if parsed:
#                 break
#         if not parsed:
#             raise FileNotFoundError("No epoch CSV or trainer_state JSON found in run_outdir. Provide logs or rerun training with logging enabled.")


In [ ]:
# CELL B: Post-session plotting (run after kernel restart; no trainer in memory)
# Paste and run this cell. It finds a run folder and plots from saved CSV/JSON.
# If auto-find picks wrong folder, set run_outdir manually.

import os, glob
from src.plot_utils import plot_loss_vs_epochs

# Option 1: paste your run folder path here (recommended)
run_outdir = None
# run_outdir = "outputs/distilbert-base-uncased_lr2e-05_bs8_ep2_20251127_123456"  # <- example, paste your actual path

# Option 2: auto-find newest folder under outputs/
if run_outdir is None:
    base = "outputs"
    candidates = [d for d in glob.glob(os.path.join(base, "*")) if os.path.isdir(d)]
    if not candidates:
        raise FileNotFoundError("No run folders found under 'outputs/'. Provide run_outdir manually.")
    run_outdir = max(candidates, key=os.path.getmtime)
    print("Auto-selected latest run folder:", run_outdir)

# Prefer loss_per_epoch.csv if present
epoch_csv = os.path.join(run_outdir, "loss_per_epoch.csv")
if os.path.exists(epoch_csv):
    print("Found loss_per_epoch.csv — plotting from that CSV:", epoch_csv)
    df = plot_loss_vs_epochs(trainer=None, hist_csv=epoch_csv, run_outdir=run_outdir, show_plot=True)
else:
    # Try trainer_state_log_history.json or trainer_state.json
    json_candidates = [
        os.path.join(run_outdir, "trainer_state_log_history.json"),
        os.path.join(run_outdir, "trainer_state.json")
    ]
    found = False
    for jpath in json_candidates:
        if os.path.exists(jpath):
            print("Found", jpath, "- parsing and plotting.")
            df = plot_loss_vs_epochs(trainer=None, hist_csv=None, run_outdir=run_outdir, show_plot=True)
            found = True
            break
    if not found:
        # fallback: try any JSON under run_outdir
        parsed = False
        for root, _, files in os.walk(run_outdir):
            for f in files:
                if f.lower().endswith(".json"):
                    cand = os.path.join(root, f)
                    try:
                        # plot_loss_vs_epochs will scan and parse files under run_outdir
                        df = plot_loss_vs_epochs(trainer=None, hist_csv=None, run_outdir=run_outdir, show_plot=True)
                        parsed = True
                        break
                    except Exception:
                        continue
            if parsed:
                break
        if not parsed:
            raise FileNotFoundError("No epoch CSV or trainer_state JSON found in run_outdir. Check the folder or re-run the in-session cell after training.")


###Cell C — Quick CSV viewer (inspect numbers)

In [ ]:
# CELL C: Quick view of saved loss_per_epoch.csv (optional)
import os, pandas as pd

# set run_outdir manually or reuse last used
try:
    run_outdir  # may be set by Cell B
except NameError:
    run_outdir = None

if run_outdir is None:
    run_outdir = input("Paste run_outdir path (or press Enter to auto-find latest under 'outputs/'): ").strip() or None

if run_outdir is None:
    import glob
    cand = glob.glob("outputs/*")
    cand = [c for c in cand if os.path.isdir(c)]
    if not cand:
        raise FileNotFoundError("No outputs folder found.")
    run_outdir = max(cand, key=os.path.getmtime)
    print("Auto-selected run_outdir:", run_outdir)

csv_path = os.path.join(run_outdir, "loss_per_epoch.csv")
if not os.path.exists(csv_path):
    print("loss_per_epoch.csv not found in", run_outdir)
else:
    df = pd.read_csv(csv_path)
    print("Loaded loss_per_epoch.csv:")
    display(df)
